In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps

/Anaconda/miniconda3/envs/asa/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [4]:
one_km = gpd.read_file('../data/clean/one_km_grid.shp')
half_km = gpd.read_file('../data/clean/half_km_grid.shp')
quarter_km = gpd.read_file('../data/clean/quarter_km_grid.shp')
print(one_km.shape)
print(half_km.shape)
print(quarter_km.shape)

(206, 20)
(398, 20)
(719, 20)


In [5]:
#check columns
one_km.columns

Index(['popE', 'p_blk16', 'p_pov16', 'p_pooh16', 'p_vac16', 'pc70_16',
       'barriers', 'crime', 'storef', 'inters', 'crime_r', 'barr_r', 'pop80',
       'p_blk80', 'p_vac80', 'p_pov80', 'pc70_80', 'pc50_16', 'pc50_80',
       'geometry'],
      dtype='object')

## Assign variables for analysis

In [97]:
# set up predictors and response matrix for one square kilometer grids
predictors1 = ['popE', 'p_blk16', 'p_pov16', 'p_pooh16', 'p_vac16','pc70_16', 'storef', 'inters', 'barriers']
response1 = 'crime'
X1_one = one_km[predictors1].dropna()
Y1_one = one_km.loc[X1_one.index][[response1]]

predictors2 = ['pop80', 'p_blk80', 'p_pov80', 'p_vac80','pc50_80', 'inters']
response2 = 'barriers'
X2_one = one_km[predictors2].dropna()
Y2_one = one_km.loc[X2_one.index][[response2]] 

# set up X and Y values for half square kilometer grids
X1_half = half_km[predictors1].dropna()
Y1_half = half_km.loc[X1_half.index][[response1]]

X2_half = half_km[predictors2].dropna()
Y2_half = half_km.loc[X2_half.index][[response2]] 

# set up X and Y values for quarter square kilometer grids
X1_quarter = quarter_km[predictors1].dropna()
Y1_quarter = quarter_km.loc[X1_quarter.index][[response1]]

X2_quarter = quarter_km[predictors2].dropna()
Y2_quarter = quarter_km.loc[X2_quarter.index][[response2]] 

In [98]:
# create spatial weights matrix for one square kilometer grids 
W1_one = ps.lib.weights.Rook.from_dataframe(one_km.loc[X1_one.index])
W1_one.transform = 'r'

W2_one = ps.lib.weights.Rook.from_dataframe(one_km.loc[X2_one.index])
W2_one.transform = 'r'

# create spatial weights matrix for half square kilometer grids 
W1_half = ps.lib.weights.Rook.from_dataframe(half_km.loc[X1_half.index])
W1_half.transform = 'r'

W2_half = ps.lib.weights.Rook.from_dataframe(half_km.loc[X2_half.index])
W2_half.transform = 'r'

# create spatial weights matrix for quarter square kilometer grids 
W1_quarter = ps.lib.weights.Rook.from_dataframe(quarter_km.loc[X1_quarter.index])
W1_quarter.transform = 'r'

W2_quarter = ps.lib.weights.Rook.from_dataframe(quarter_km.loc[X2_quarter.index])
W2_quarter.transform = 'r'

## Models

### One square kilometer grids

#### Crime Model

In [99]:
# ordinary least squares
ols1_one = ps.model.spreg.OLS(y=Y1_one.values, x=X1_one.values, w=W1_one, name_w='rook', name_x=X1_one.columns.tolist(),
                          name_y=response1, name_ds='one_km', spat_diag=True, moran=True)
print(ols1_one.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         206
Mean dependent var  :     27.7524                Number of Variables   :          10
S.D. dependent var  :     33.3913                Degrees of Freedom    :         196
R-squared           :      0.6675
Adjusted R-squared  :      0.6523
Sum squared residual:   75990.759                F-statistic           :     43.7270
Sigma-square        :     387.708                Prob(F-statistic)     :   2.898e-42
S.E. of regression  :      19.690                Log likelihood        :    -901.082
Sigma-square ML     :     368.887                Akaike info criterion :    1822.164
S.E of regression ML:     19.2064                Schwarz criterion     :    1855.443

-----------------------------------------------------------------------------

In [100]:
# Spatial lag model - GMM
gmm1_one = ps.model.spreg.GM_Lag(y=Y1_one.values, x=X1_one.values, w=W1_one, name_w='rook', name_x=X1_one.columns.tolist(),
                          name_y=response1, name_ds='one_km', robust='white', spat_diag=True)
print(gmm1_one.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         206
Mean dependent var  :     27.7524                Number of Variables   :          11
S.D. dependent var  :     33.3913                Degrees of Freedom    :         195
Pseudo R-squared    :      0.7222
Spatial Pseudo R-squared:  0.6786

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT     -28.8585347       6.0862887      -4.7415652       0.0000021
                popE       0.0087927       0.0025202       3.4889312       0.0004850
             p_blk16       0.2123772 

#### Barrier Models

In [101]:
ols2_one = ps.model.spreg.OLS(y=Y2_one.values, x=X2_one.values, w=W2_one, name_w='rook', name_x=X2_one.columns.tolist(),
                          name_y=response2, name_ds='one_km', spat_diag=True, moran=True)
print(ols2_one.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           7
S.D. dependent var  :      2.7753                Degrees of Freedom    :         198
R-squared           :      0.3172
Adjusted R-squared  :      0.2965
Sum squared residual:    1072.928                F-statistic           :     15.3280
Sigma-square        :       5.419                Prob(F-statistic)     :   2.098e-14
S.E. of regression  :       2.328                Log likelihood        :    -460.534
Sigma-square ML     :       5.234                Akaike info criterion :     935.068
S.E of regression ML:      2.2877                Schwarz criterion     :     958.329

-----------------------------------------------------------------------------

In [102]:
# Spatial lag model - GMM
gmm2_one = ps.model.spreg.GM_Lag(y=Y2_one.values, x=X2_one.values, w=W2_one, name_w='rook', name_x=X2_one.columns.tolist(),
                          name_y=response2, name_ds='one_km', robust='white', spat_diag=True)
print(gmm2_one.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           8
S.D. dependent var  :      2.7753                Degrees of Freedom    :         197
Pseudo R-squared    :      0.4730
Spatial Pseudo R-squared:  0.3066

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -1.2269328       0.3354569      -3.6574976       0.0002547
               pop80       0.0002213       0.0001695       1.3058274       0.1916113
             p_blk80       0.0033599 

In [103]:
# Spatial error model - GMM assuming heteroskedasticity
gmm3_one = ps.model.spreg.GM_Error_Het(y=Y2_one.values, x=X2_one.values, w=W2_one, name_w='rook', name_x=X2_one.columns.tolist(),
                          name_y=response2, name_ds='one_km')
print(gmm3_one.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           7
S.D. dependent var  :      2.7753                Degrees of Freedom    :         198
Pseudo R-squared    :      0.3064
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -1.4152849       0.3795382      -3.7289654       0.0001923
               pop80       0.0003369       0.0002137       1.5765321       0.11

### Half square kilometer grids

### Crime

In [104]:
# ordinary least squares
ols1_half = ps.model.spreg.OLS(y=Y1_half.values, x=X1_half.values, w=W1_half, name_w='rook', name_x=X1_half.columns.tolist(),
                          name_y=response1, name_ds='half_km', spat_diag=True, moran=True)
print(ols1_half.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     half_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         395
Mean dependent var  :     14.4633                Number of Variables   :          10
S.D. dependent var  :     18.4447                Degrees of Freedom    :         385
R-squared           :      0.6076
Adjusted R-squared  :      0.5985
Sum squared residual:   52592.155                F-statistic           :     66.2504
Sigma-square        :     136.603                Prob(F-statistic)     :    9.52e-73
S.E. of regression  :      11.688                Log likelihood        :   -1526.539
Sigma-square ML     :     133.145                Akaike info criterion :    3073.079
S.E of regression ML:     11.5388                Schwarz criterion     :    3112.868

-----------------------------------------------------------------------------

In [105]:
# Spatial lag model - GMM
gmm1_half = ps.model.spreg.GM_Lag(y=Y1_half.values, x=X1_half.values, w=W1_half, name_w='rook', name_x=X1_half.columns.tolist(),
                          name_y=response1, name_ds='half_km', robust='white', spat_diag=True)
print(gmm1_half.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     half_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         395
Mean dependent var  :     14.4633                Number of Variables   :          11
S.D. dependent var  :     18.4447                Degrees of Freedom    :         384
Pseudo R-squared    :      0.6898
Spatial Pseudo R-squared:  0.6218

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT     -11.8147413       2.3514017      -5.0245526       0.0000005
                popE       0.0063186       0.0017002       3.7163664       0.0002021
             p_blk16       0.1023980 

### Barriers

In [106]:
ols2_half = ps.model.spreg.OLS(y=Y2_half.values, x=X2_half.values, w=W2_half, name_w='rook', name_x=X2_half.columns.tolist(),
                          name_y=response2, name_ds='half_km', spat_diag=True, moran=True)
print(ols2_half.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     half_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         392
Mean dependent var  :      0.7117                Number of Variables   :           7
S.D. dependent var  :      1.7242                Degrees of Freedom    :         385
R-squared           :      0.2377
Adjusted R-squared  :      0.2259
Sum squared residual:     886.079                F-statistic           :     20.0121
Sigma-square        :       2.302                Prob(F-statistic)     :   2.222e-20
S.E. of regression  :       1.517                Log likelihood        :    -716.071
Sigma-square ML     :       2.260                Akaike info criterion :    1446.141
S.E of regression ML:      1.5035                Schwarz criterion     :    1473.940

-----------------------------------------------------------------------------

In [107]:
# Spatial lag model - GMM
gmm2_half = ps.model.spreg.GM_Lag(y=Y2_half.values, x=X2_half.values, w=W2_half, name_w='rook', name_x=X2_half.columns.tolist(),
                          name_y=response2, name_ds='half_km', robust='white', spat_diag=True)
print(gmm2_half.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     half_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         392
Mean dependent var  :      0.7117                Number of Variables   :           8
S.D. dependent var  :      1.7242                Degrees of Freedom    :         384
Pseudo R-squared    :      0.3720
Spatial Pseudo R-squared:  0.2371

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.6929211       0.1933121      -3.5844690       0.0003378
               pop80       0.0004144       0.0001666       2.4867387       0.0128920
             p_blk80       0.0018727 

In [108]:
# Spatial error model - GMM assuming heteroskedasticity
gmm3_half = ps.model.spreg.GM_Error_Het(y=Y2_half.values, x=X2_half.values, w=W2_half, name_w='rook', name_x=X2_half.columns.tolist(),
                          name_y=response2, name_ds='half_km')
print(gmm3_half.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     half_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         392
Mean dependent var  :      0.7117                Number of Variables   :           7
S.D. dependent var  :      1.7242                Degrees of Freedom    :         385
Pseudo R-squared    :      0.2333
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.8270284       0.1895425      -4.3632871       0.0000128
               pop80       0.0005544       0.0001817       3.0509205       0.00

### Quarter square kilometer grids

### Crime

In [109]:
# ordinary least squares
ols1_quarter = ps.model.spreg.OLS(y=Y1_quarter.values, x=X1_quarter.values, w=W1_quarter, name_w='rook', name_x=X1_quarter.columns.tolist(),
                          name_y=response1, name_ds='quarter_km', spat_diag=True, moran=True)
print(ols1_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  quarter_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         715
Mean dependent var  :      7.9902                Number of Variables   :          10
S.D. dependent var  :     10.4367                Degrees of Freedom    :         705
R-squared           :      0.5247
Adjusted R-squared  :      0.5187
Sum squared residual:   36963.591                F-statistic           :     86.4833
Sigma-square        :      52.431                Prob(F-statistic)     :  1.014e-107
S.E. of regression  :       7.241                Log likelihood        :   -2425.024
Sigma-square ML     :      51.697                Akaike info criterion :    4870.047
S.E of regression ML:      7.1901                Schwarz criterion     :    4915.770

-----------------------------------------------------------------------------

In [110]:
# Spatial lag model - GMM
gmm1_quarter = ps.model.spreg.GM_Lag(y=Y1_quarter.values, x=X1_quarter.values, w=W1_quarter, name_w='rook', name_x=X1_quarter.columns.tolist(),
                          name_y=response1, name_ds='quarter_km', robust='white', spat_diag=True)
print(gmm1_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :  quarter_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         715
Mean dependent var  :      7.9902                Number of Variables   :          11
S.D. dependent var  :     10.4367                Degrees of Freedom    :         704
Pseudo R-squared    :      0.6073
Spatial Pseudo R-squared:  0.5338

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -6.9277696       1.4181142      -4.8851987       0.0000010
                popE       0.0084124       0.0015939       5.2778921       0.0000001
             p_blk16       0.0676242 

In [111]:
# Spatial error model - GMM assuming heteroskedasticity
gmm3_quarter = ps.model.spreg.GM_Error_Het(y=Y2_quarter.values, x=X2_quarter.values, w=W2_quarter, name_w='rook', name_x=X2_quarter.columns.tolist(),
                          name_y=response2, name_ds='quarter_km')
print(gmm3_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :  quarter_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         708
Mean dependent var  :      0.3941                Number of Variables   :           7
S.D. dependent var  :      1.0470                Degrees of Freedom    :         701
Pseudo R-squared    :      0.1620
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.4392680       0.0796553      -5.5146138       0.0000000
               pop80       0.0004741       0.0001442       3.2884605       0.00

### Barriers

In [112]:
ols2_quarter = ps.model.spreg.OLS(y=Y2_quarter.values, x=X2_quarter.values, w=W2_quarter, name_w='rook', name_x=X2_quarter.columns.tolist(),
                          name_y=response2, name_ds='quarter_km', spat_diag=True, moran=True)
print(ols2_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :  quarter_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         708
Mean dependent var  :      0.3941                Number of Variables   :           7
S.D. dependent var  :      1.0470                Degrees of Freedom    :         701
R-squared           :      0.1635
Adjusted R-squared  :      0.1563
Sum squared residual:     648.369                F-statistic           :     22.8283
Sigma-square        :       0.925                Prob(F-statistic)     :   1.159e-24
S.E. of regression  :       0.962                Log likelihood        :    -973.462
Sigma-square ML     :       0.916                Akaike info criterion :    1960.924
S.E of regression ML:      0.9570                Schwarz criterion     :    1992.861

-----------------------------------------------------------------------------

In [113]:
# Spatial lag model - GMM
gmm2_quarter = ps.model.spreg.GM_Lag(y=Y2_quarter.values, x=X2_quarter.values, w=W2_quarter, name_w='rook', name_x=X2_quarter.columns.tolist(),
                          name_y=response2, name_ds='quarter_km', robust='white', spat_diag=True)
print(gmm2_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :  quarter_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         708
Mean dependent var  :      0.3941                Number of Variables   :           8
S.D. dependent var  :      1.0470                Degrees of Freedom    :         700
Pseudo R-squared    :      0.2571
Spatial Pseudo R-squared:  0.1246

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.1605463       0.1193633      -1.3450227       0.1786179
               pop80       0.0001188       0.0001462       0.8126465       0.4164207
             p_blk80       0.0001295 

## Summary

### OLS Crime Models

In [114]:
print(ols1_one.summary)
print(ols1_half.summary)
print(ols1_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         206
Mean dependent var  :     27.7524                Number of Variables   :          10
S.D. dependent var  :     33.3913                Degrees of Freedom    :         196
R-squared           :      0.6675
Adjusted R-squared  :      0.6523
Sum squared residual:   75990.759                F-statistic           :     43.7270
Sigma-square        :     387.708                Prob(F-statistic)     :   2.898e-42
S.E. of regression  :      19.690                Log likelihood        :    -901.082
Sigma-square ML     :     368.887                Akaike info criterion :    1822.164
S.E of regression ML:     19.2064                Schwarz criterion     :    1855.443

-----------------------------------------------------------------------------

### GMM Lag Crime Models

In [115]:
print(gmm1_one.summary)
print(gmm1_half.summary)
print(gmm1_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :       crime                Number of Observations:         206
Mean dependent var  :     27.7524                Number of Variables   :          11
S.D. dependent var  :     33.3913                Degrees of Freedom    :         195
Pseudo R-squared    :      0.7222
Spatial Pseudo R-squared:  0.6786

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT     -28.8585347       6.0862887      -4.7415652       0.0000021
                popE       0.0087927       0.0025202       3.4889312       0.0004850
             p_blk16       0.2123772 

### OLS Barrier Models

In [116]:
print(ols2_one.summary)
print(ols2_half.summary)
print(ols2_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           7
S.D. dependent var  :      2.7753                Degrees of Freedom    :         198
R-squared           :      0.3172
Adjusted R-squared  :      0.2965
Sum squared residual:    1072.928                F-statistic           :     15.3280
Sigma-square        :       5.419                Prob(F-statistic)     :   2.098e-14
S.E. of regression  :       2.328                Log likelihood        :    -460.534
Sigma-square ML     :       5.234                Akaike info criterion :     935.068
S.E of regression ML:      2.2877                Schwarz criterion     :     958.329

-----------------------------------------------------------------------------

### GMM Lag Barrier Models

In [117]:
print(gmm2_one.summary)
print(gmm2_half.summary)
print(gmm2_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           8
S.D. dependent var  :      2.7753                Degrees of Freedom    :         197
Pseudo R-squared    :      0.4730
Spatial Pseudo R-squared:  0.3066

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -1.2269328       0.3354569      -3.6574976       0.0002547
               pop80       0.0002213       0.0001695       1.3058274       0.1916113
             p_blk80       0.0033599 

### GMM Het Error Barrier Models

In [118]:
print(gmm3_one.summary)
print(gmm3_half.summary)
print(gmm3_quarter.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :      one_km
Weights matrix      :        rook
Dependent Variable  :    barriers                Number of Observations:         205
Mean dependent var  :      1.3610                Number of Variables   :           7
S.D. dependent var  :      2.7753                Degrees of Freedom    :         198
Pseudo R-squared    :      0.3064
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -1.4152849       0.3795382      -3.7289654       0.0001923
               pop80       0.0003369       0.0002137       1.5765321       0.11